In [53]:
# Import des bibliothèques
print("Importation des bibliothèques...")

# 1. Google Earth Engine
try:
    import ee
    print("✓ Google Earth Engine importé")
except Exception as e:
    print(f"✗ Erreur avec Google Earth Engine: {e}")

# 2. Analyse de données
try:
    import pandas as pd
    print("✓ pandas importé (pour les tableaux)")
except Exception as e:
    print(f"✗ Erreur avec pandas: {e}")

# 3. Calcul numérique
try:
    import numpy as np
    print("✓ numpy importé (pour les calculs)")
except Exception as e:
    print(f"✗ Erreur avec numpy: {e}")

# 4. Visualisation
try:
    import matplotlib.pyplot as plt
    print("✓ matplotlib importé (pour les graphiques)")
except Exception as e:
    print(f"✗ Erreur avec matplotlib: {e}")

print("\nToutes les bibliothèques sont importées !")

Importation des bibliothèques...
✓ Google Earth Engine importé
✓ pandas importé (pour les tableaux)
✓ numpy importé (pour les calculs)
✓ matplotlib importé (pour les graphiques)

Toutes les bibliothèques sont importées !


In [55]:
# Ton project ID exact
TON_PROJET = "userscheikhthioub501"  # Ton projet
print(f"Project ID utilisé : {TON_PROJET}")

print("\nÉtape 1 : Importation de Google Earth Engine...")
import ee

print("Étape 2 : Initialisation...")
try:
    # Initialiser avec TON projet
    ee.Initialize(project=TON_PROJET)
    print("✓ SUCCÈS ! Google Earth Engine est initialisé")
    print(f"✓ Projet utilisé : {TON_PROJET}")
    
except Exception as e:
    print(f"✗ ERREUR : {e}")
    print("\nEssayons sans spécifier de projet...")
    try:
        ee.Initialize()
        print("✓ SUCCÈS avec initialisation simple !")
    except Exception as e2:
        print(f"✗ Échec aussi : {e2}")

Project ID utilisé : userscheikhthioub501

Étape 1 : Importation de Google Earth Engine...
Étape 2 : Initialisation...
✓ SUCCÈS ! Google Earth Engine est initialisé
✓ Projet utilisé : userscheikhthioub501


In [56]:




# Section 1




#  1 Chargeons la collection ECMWF/ERA5/DAILY puis selectionnons la bande total_precipitation
era5_daily = ee.ImageCollection('ECMWF/ERA5/DAILY') \
    .select('total_precipitation')  # Sélection de la bande précipitations

#  3 définissons la géométrie du Ghana via FAO/GAUL
countries = ee.FeatureCollection("FAO/GAUL/2015/level0")
ghana = countries.filter(ee.Filter.eq('ADM0_NAME', 'Ghana'))


# 4 Restreignons la période à 2015-01-01 -> 2020-12-31
era5_ghana = era5_daily.filterDate('2015-01-01', '2020-12-31') \
                       .filterBounds(ghana)

#   5 Convertissons les précipitations journalières en précipitations annuelles moyennes

def add_year(image):
    year = ee.Date(image.get('system:time_start')).get('year')
    return image.set('year', year)

# Ajouter l'année à chaque image
era5_ghana = era5_ghana.map(add_year)

# Calculer la moyenne annuelle
years = ee.List.sequence(2015, 2020)

def yearly_mean(year):
    year = ee.Number(year)
    # Filtrer pour l'année
    yearly_imgs = era5_ghana.filter(ee.Filter.eq('year', year))
    # Moyenne de l'année
    mean_image = yearly_imgs.mean()
    # Ajouter l'année en propriété
    return mean_image.set('year', year)

annual_mean_collection = ee.ImageCollection(years.map(yearly_mean))

#  Afficher les résultats
print('Collection de précipitations annuelles moyennes:', annual_mean_collection.getInfo())

print('fin de la section 1 ')



Collection de précipitations annuelles moyennes: {'type': 'ImageCollection', 'bands': [], 'features': [{'type': 'Image', 'bands': [{'id': 'total_precipitation', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'year': 2015, 'system:index': '0'}}, {'type': 'Image', 'bands': [{'id': 'total_precipitation', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'year': 2016, 'system:index': '1'}}, {'type': 'Image', 'bands': [{'id': 'total_precipitation', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'year': 2017, 'system:index': '2'}}, {'type': 'Image', 'bands': [{'id': 'total_precipitation', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'year': 2018, 'system:inde

In [68]:

# Section 2 



era5_mm = era5_ghana.map(lambda img: 
    img.multiply(1000).copyProperties(img, ["system:time_start"])
)


# pour éviter que la saison sèche tire artificiellement la moyenne vers le bas considérons  les jours avec précipitations (> 0 mm)
era5_bony = era5_mm.map(
    lambda img: img.updateMask(img.gt(0))
)


#Ajouter l'année comme propriété

def add_year(image):
    year = ee.Date(image.get("system:time_start")).get("year")
    return image.set("year", year)

era5_bony = era5_bony.map(add_year)


#  Calcul des statistiques PAR ANNÉE

years = ee.List.sequence(2015, 2020)

def annual_stats(year):
    year = ee.Number(year)
    
    yearly_imgs = era5_bony.filter(ee.Filter.eq("year", year))
    
    # Moyenne spatiale annuelle (jours pluvieux uniquement)
    mean_annual = yearly_imgs.mean()
    
    mean_val = mean_annual.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=ghana,
        scale=10000,
        maxPixels=1e13
    ).get("total_precipitation")
    
    var_val = mean_annual.reduceRegion(
        reducer=ee.Reducer.variance(),
        geometry=ghana,
        scale=10000,
        maxPixels=1e13
    ).get("total_precipitation")
    
    std_val = ee.Number(var_val).sqrt()
    cv_val = std_val.divide(mean_val)
    
    return ee.Feature(
        None,
        {
            "year": year,
            "mean_mm": mean_val,
            "variance_mm2": var_val,
            "cv": cv_val
        }
    )

annual_stats_fc = ee.FeatureCollection(years.map(annual_stats))

print("Statistiques annuelles  :")
print(annual_stats_fc.getInfo())


#  Statistiques moyennes sur la période 2015–2020

mean_5y = annual_stats_fc.aggregate_mean("mean_mm")
var_5y  = annual_stats_fc.aggregate_mean("variance_mm2")
cv_5y   = annual_stats_fc.aggregate_mean("cv")


print("Moyenne nationale (mm) :", mean_5y.getInfo()*1000)
print("Variance nationale (mm²) :", var_5y.getInfo()*1000000)
print("Coefficient de variation :", cv_5y.getInfo())


Statistiques annuelles (jours pluvieux uniquement) :
{'type': 'FeatureCollection', 'columns': {'cv': 'Float', 'mean_mm': 'Float', 'system:index': 'String', 'variance_mm2': 'Float', 'year': 'Float'}, 'features': [{'type': 'Feature', 'geometry': None, 'id': '0', 'properties': {'cv': 0.18374025752347758, 'mean_mm': 3.184761377799048, 'variance_mm2': 0.3424226130969228, 'year': 2015}}, {'type': 'Feature', 'geometry': None, 'id': '1', 'properties': {'cv': 0.21041765013407687, 'mean_mm': 3.6991409618973465, 'variance_mm2': 0.6058513706996502, 'year': 2016}}, {'type': 'Feature', 'geometry': None, 'id': '2', 'properties': {'cv': 0.22762269033903249, 'mean_mm': 3.442270005474691, 'variance_mm2': 0.6139329876693651, 'year': 2017}}, {'type': 'Feature', 'geometry': None, 'id': '3', 'properties': {'cv': 0.17224711088191838, 'mean_mm': 4.0644868169090955, 'variance_mm2': 0.490134565239738, 'year': 2018}}, {'type': 'Feature', 'geometry': None, 'id': '4', 'properties': {'cv': 0.18969274466534017, 'mea

In [70]:
#  2 Visuqliwqtions

import  folium

def add_ee_layer(self, ee_image_object, vis_params, name):
    """Ajouterons une image Earth Engine à une carte Folium."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles = map_id_dict['tile_fetcher'].url_format,
        attr = "Google Earth Engine",
        name = name,
        overlay = True,
        control = True
    ).add_to(self)

# Ajouter la méthode à folium.Map
folium.Map.add_ee_layer = add_ee_layer

# Créer une carte centrée sur le Ghana
ghana_center = [7.9465, -1.0232]
my_map = folium.Map(location=ghana_center, zoom_start=6)

# Paramètres de visualisation
vis_params = {
    'min': 0,
    'max': 0.006, # pris pour couvrir tout le pays
    'palette': ['yellow', 'green', 'blue']
}

# Sélectionner la moyenne sur toute la période 2015-2020
mean_image = annual_mean_collection.mean().clip(ghana)

# Ajouter l'image à la carte
my_map.add_ee_layer(mean_image, vis_params, 'Précipitations moyennes 2015-2020')


# Afficher la carte
my_map


In [72]:
 '''  3 Commentaires sur l'analyse spatiale

#  Distribution spatiale
# - Nous observons tout d4abord que les zones du nord sont  moins pluvieuses. Ensuite les zones du sud sont plus pluvieuses. 

# Les zones côtières ont des précipitations modérées à élevées
 
#  Le coefficient de variation de 0.28 permet dès lors de mettre en exuergue l’hétérogénéité spatiale'''



"  Commentaires sur l'analyse spatiale\n\n#  Distribution spatiale\n# - Nous observons tout d4abord que les zones du nord sont  moins pluvieuses. Ensuite les zones du sud sont plus pluvieuses. \n\n# Les zones côtières ont des précipitations modérées à élevées\n\n#  Le coefficient de variation de 0.28 permet dès lors de mettre en exuergue l’hétérogénéité spatiale"

In [74]:
# Questions coonceptuelles

''' 1 Masquage de l’information par la moyenne nationale
La moyenne nationale donne un chiffre global (0.003 m/an) mais ne montre pas les différences entre les régions le chiffre concerne tout les 
pays et non les régions ainsi l'heterogeneité entre le nord et le sud ne peux etre vu  
  entre nord, sud et côte.


 2  Les cartes permttent par exemple d'identifier d’identifier les zones sensibles à la sécheresse ou aux fortes pluies ce 
 qui est indispensbale pour  pour l’agriculture, la planification des ressources en eau et la gestion du risque climatique'''


"Masquage de l’information par la moyenne nationale\nLa moyenne nationale donne un chiffre global (0.003 m/an) mais ne montre pas les différences entre les régions le chiffre concerne tout les \npays et non les régions ainsi l'heterogeneité entre le nord et le sud ne peux etre vu  \n  entre nord, sud et côte.\n\n\n Les cartes permttent par exemple d'identifier d’identifier les zones sensibles à la sécheresse ou aux fortes pluies ce \n qui est indispensbale pour  pour l’agriculture, la planification des ressources en eau et la gestion du risque climatique"

In [76]:
# Section 3



era5_daily = ee.ImageCollection('ECMWF/ERA5/DAILY').select('total_precipitation')

# Filtrer la période 2015-2020
era5_ghana = era5_daily.filterDate('2015-01-01', '2020-12-31').filterBounds(ghana)

# Calculer la moyenne annuelle sur toute la période
mean_precip = era5_ghana.mean().clip(ghana)

#   1 Construisons une grille spatiale régulière sur le Ghana

# On va condtruire une grille carrée d'environ 0.25° x 0.25° (~25 km)
def create_grid(region, scale=0.25):
    bounds = region.geometry().bounds().getInfo()['coordinates'][0]
    lon_min = bounds[0][0]
    lat_min = bounds[0][1]
    lon_max = bounds[2][0]
    lat_max = bounds[2][1]

    lon_steps = int((lon_max - lon_min) / scale)
    lat_steps = int((lat_max - lat_min) / scale)

    features = []
    for i in range(lon_steps):
        for j in range(lat_steps):
            x0 = lon_min + i * scale
            y0 = lat_min + j * scale
            x1 = x0 + scale
            y1 = y0 + scale
            poly = ee.Geometry.Rectangle([x0, y0, x1, y1])
            features.append(ee.Feature(poly))
    return ee.FeatureCollection(features)

grid = create_grid(ghana, scale=0.25)

# 2 Associons donc  à chaque cellule la valeur moyenne de précipitations

def add_precip_to_cell(feature):
    mean_value = mean_precip.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=feature.geometry(),
        scale=10000
    ).get('total_precipitation')
    return feature.set('precip', mean_value)

grid_precip = grid.map(add_precip_to_cell)


#  3 Calculons   l'indicateur global d'autocorrélation spatiale 

''' On utilise une matrice de voisinage de type "rook" voisins partageant un côté comme aux echecs 
 Justifions : La structure "rook" capture les influences immédiates nord/sud/est/ouest et est adaptée à une grille carrée régulière.'''
def Moran(feature_collection, attr):
    # Extraction ddes valeurs
    values = feature_collection.aggregate_array(attr)
    n = len(values.getInfo())
    mean_val = sum(values.getInfo()) / n

    # Calculer les écarts
    dev = [v - mean_val for v in values.getInfo()]

    # Matrice de poids rook
    # Ici on simplifie : poids = 1 si voisin immédiat (N, S, E, O), sinon 0
    
    W = []
    for i, v in enumerate(values.getInfo()):
        # Compter les voisins (approximation)
        W.append(4)  # chaque cellule a 4 voisins en "rook" sauf bordure

    # Calcul Moran I simplifié (formule globale)
    num = 0
    for i in range(n):
        for j in range(n):
            if i != j:
                num += dev[i] * dev[j] * 1  # poids 1 pour voisins, 0 sinon
    denom = sum([d**2 for d in dev])
    I = (n / sum(W)) * (num / denom)
    return I

# Calcul 
indice = Moran(grid_precip, 'precip')
print("Indice global d'autocorrélation spatiale  :", indice)


# 4 Interprétation

''' - I < 0 par conséquent il y a une  dispersion spatiale (zones sèches proches de zones humides)
# - Magnitude : on sait que  plus I est proche de 1, plus il ya une forte autocorrélation positive et plus I est 
 proche de -1 il ya une  forte autocorrélation négative mais dans notre cas les valeurs sont un peu proche de 0 donc 
 il n'y a pas une nette structure spatiale 

'''


Indice global d'autocorrélation spatiale  : -0.2500000000000002


" - I < 0 par conséquent il y a une  dispersion spatiale (zones sèches proches de zones humides)\n# - Magnitude : on sait que  plus I est proche de 1, plus il ya une forte autocorrélation positive et plus I est \n proche de -1 il ya une  forte autocorrélation négative mais dans notre cas les valeurs sont un peu proche de 0 donc \n il n'y a pas une nette structure spatiale \n\n"

In [77]:

# 5 Visualisation des zones de regroupement

import folium


def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles = map_id_dict['tile_fetcher'].url_format,
        attr = "Google Earth Engine",
        name = name,
        overlay = True,
        control = True
    ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

# Carte centrée sur le Ghana
my_map = folium.Map(location=[7.9465, -1.0232], zoom_start=6)

# Visualiser la grille des précipitations
# Convertir grid avec valeur precip en image (approximation)
grid_image = grid_precip.reduceToImage(properties=['precip'], reducer=ee.Reducer.mean())

vis_params = {'min':0,'max':0.005,'palette':['yellow','green','blue']}
my_map.add_ee_layer(grid_image, vis_params, 'Grille précipitations moyennes 2015-2020')
my_map.add_child(folium.LayerControl())

'''
 la zone du nord est caractérisée par des  précipitations plus faibles donc :  regroupement de valeurs basses
 
 la zones du sud est cartérisée par des  précipitations plus élevées  donc : regroupement de valeurs hautes
 
Les zones côtières dont intermédiaires, plus homogènes

La grille et l'indice Moran permettent d'identifier ces clusters


# - Justification de la matrice "rook" : considère uniquement les voisins immédiats par suite    ce qui est cohérent pour analyser la continuité des précipitations sur la grille

'''


'\n la zone du nord est caractérisée par des  précipitations plus faibles donc :  regroupement de valeurs basses\n \n la zones du sud est cartérisée par des  précipitations plus élevées  donc : regroupement de valeurs hautes\n \nLes zones côtières dont intermédiaires, plus homogènes\n\nLa grille et l\'indice Moran permettent d\'identifier ces clusters\n\n\n# - Justification de la matrice "rook" : considère uniquement les voisins immédiats par suite    ce qui est cohérent pour analyser la continuité des précipitations sur la grille\n\n'

In [78]:
# Section 5 

''' 1 Reponse sur ERA

  ERA5 est une reanalyse globale, conçue pour des tendances régionales et globales. Au Ghana, où le climat  peut varier rapidement entre zones côtières, le nord et le sud, cette résolution masque les variations locales fines.

Parailleurs il y'a le souci des paramètres des  modèles physiques : Les données ERA5 sont générées par assimilation de modèles numériques et observations

satellitaires, ce qui peut entraîner des biais locaux .

Aussi  les précipitations sont déjà agrégées et lissées, ce qui atténue les extrêmes, essentiels pour l’agriculture et la gestion des risques

climatiques.

Ainsi ERA5 est bon pour identifier des tendances générales, mais inadapté pour une analyse spatiale très fine (<5 km) dans des zones à hétérogénéité 

climatique.'''

" 1 Reponse sur ERA\n\n  ERA5 est une reanalyse globale, conçue pour des tendances régionales et globales. Au Ghana, où le climat  peut varier rapidement entre zones côtières, le nord et le sud, cette résolution masque les variations locales fines.\n\nParailleurs il y'a le souci des paramètres des  modèles physiques : Les données ERA5 sont générées par assimilation de modèles numériques et observations\n\nsatellitaires, ce qui peut entraîner des biais locaux .\n\nAussi  les précipitations sont déjà agrégées et lissées, ce qui atténue les extrêmes, essentiels pour l’agriculture et la gestion des risques\n\nclimatiques.\n\nAinsi ERA5 est bon pour identifier des tendances générales, mais inadapté pour une analyse spatiale très fine (<5 km) dans des zones à hétérogénéité \n\nclimatique."

In [ ]:
''' 2 Impact de la résolution spatiale sur les résultats

d'abord abordons le fait qu'une grille de 25 km ne capture pas les différences locales entre zones côtières, savanes et forêts. 

La moyenne nationale ou régionale peut donner l’impression d’homogénéité

alors que le terrain est très hétérogène.

ensuite il y'a l'autocorrélation spatiale: Des cellules larges réduisent la variance locale et augmentent artificiellement l’autocorrélation globale  

parce que les valeurs sont moyennées sur une grande surface.

enfin l'identification des zones de regroupement : Les clusters locaux de fortes ou faibles précipitations peuvent disparaître, ce qui masque les zones

sensibles à la sécheresse ou aux inondations.

 Donc la résolution influence directement la détection des patterns spatiaux et la précision des analyses statistiques.'''

In [ ]:
''' 3
On peut utiliser des données alternatives et  méthodes complémentaires par exemple  :

-Utiliser des données satellitaires à haute résolution pour mieux capturer les variations locales.

-Mettre en place  des  zones de controle et stations locales pour ajuster et valider les données reanalysées

Calculer des indices d’hétérogénéité spatiale locale  pour détecter les clusters précis.

Utiliser des grilles adaptatives : cellules plus petites dans les zones à forte variabilité (sud montagneux, côtes) et plus grandes dans les 
zones homogènes (nord).
